# Introduction

This notebook demonstrates the usage of `polars_h3`, a plugin extending the Polars DataFrame API with H3 geospatial functions.

## What is H3?
H3 is a hierarchical hexagonal geospatial indexing system. It assigns indexes to the surface of the Earth in a hexagonal grid at multiple resolutions. With `polars_h3`, you can integrate H3 operations directly into Polars workflows.

## Prerequisites
- Polars
- polars_h3
- H3 Python bindings

Make sure these are installed in your environment.


In [ ]:
# If needed, uncomment to install
# !pip install polars polars_h3 h3

In [2]:
import polars as pl
import polars_h3 as plh3


# Convert Latitude/Longitude to H3 Cell

We can convert a given latitude and longitude to an H3 cell at a specified resolution.


In [3]:
df = pl.DataFrame(
    {"latitude": [37.7752702151959], "longitude": [-122.418307270836]}
).with_columns(h3_cell=plh3.latlng_to_cell("latitude", "longitude", 9))

df


latitude,longitude,h3_cell
f64,f64,u64
37.77527,-122.418307,617700169958293503


# Convert H3 Cell Back to Latitude/Longitude

We can also retrieve the coordinates for the center of an H3 cell.


In [5]:
df_coords = df.with_columns(
    lat=plh3.cell_to_lat("h3_cell"), lng=plh3.cell_to_lng("h3_cell")
)

df_coords


latitude,longitude,h3_cell,lat,lng
f64,f64,u64,f64,f64
37.77527,-122.418307,617700169958293503,37.776702,-122.418459


# Check Validity and Resolution of H3 Cells

Use `is_valid_cell` and `get_resolution` to inspect cells.


In [6]:
df_inspect = df.with_columns(
    is_valid=plh3.is_valid_cell("h3_cell"),
    resolution=plh3.get_resolution("h3_cell"),
)

df_inspect


latitude,longitude,h3_cell,is_valid,resolution
f64,f64,u64,bool,u32
37.77527,-122.418307,617700169958293503,true,9


# Converting Between Integer and String H3 Representations

H3 cells can be stored as 64-bit integers or hex strings. `int_to_str` and `str_to_int` can help convert between these.


In [7]:
df_int_str = df.with_columns(
    h3_str=plh3.int_to_str("h3_cell"),
    h3_int=plh3.str_to_int(plh3.int_to_str("h3_cell")),
)

df_int_str


latitude,longitude,h3_cell,h3_str,h3_int
f64,f64,u64,str,u64
37.77527,-122.418307,617700169958293503,"""8928308280fffff""",617700169958293503


# Finding Neighbors

Given an H3 cell, we can find its neighbors at the same resolution.


In [8]:
df_neighbors = df.with_columns(
    neighbors=plh3.grid_disk("h3_cell", 1)  # radius=1 ring around the cell
)

df_neighbors


latitude,longitude,h3_cell,neighbors
f64,f64,u64,list[u64]
37.77527,-122.418307,617700169958293503,"[617700169958293503, 617700169958031359, … 617700169957507071]"


# Directed Edges

`polars_h3` lets you find directed edges between cells and perform operations on them.


In [9]:
# Example: Convert a cell to its directed edges, then convert those edges back to their origin/destination.
df_edges = df.with_columns(edges=plh3.origin_to_directed_edges("h3_cell"))

df_edges_explored = df_edges.explode("edges").with_columns(
    origin=plh3.get_directed_edge_origin("edges"),
    destination=plh3.get_directed_edge_destination("edges"),
)

df_edges_explored


latitude,longitude,h3_cell,edges,origin,destination
f64,f64,u64,u64,u64,u64
37.77527,-122.418307,617700169958293503,1266218516299644927,617700169958293503,617700169961177087
37.77527,-122.418307,617700169958293503,1338276110337572863,617700169958293503,617700169964847103
37.77527,-122.418307,617700169958293503,1410333704375500799,617700169958293503,617700169965109247
37.77527,-122.418307,617700169958293503,1482391298413428735,617700169958293503,617700169957507071
37.77527,-122.418307,617700169958293503,1554448892451356671,617700169958293503,617700169957769215
37.77527,-122.418307,617700169958293503,1626506486489284607,617700169958293503,617700169958031359


# Geospatial Metrics: Distances and Areas

We can compute great circle distances between two lat/lng points and compute areas of H3 cells.


In [10]:
df_metrics = pl.DataFrame(
    {
        "lat1": [40.7128],
        "lng1": [-74.0060],  # New York
        "lat2": [42.3601],
        "lng2": [-71.0589],  # Boston
    }
).with_columns(
    distance_km=plh3.great_circle_distance("lat1", "lng1", "lat2", "lng2", "km"),
    distance_m=plh3.great_circle_distance("lat1", "lng1", "lat2", "lng2", "m"),
)

df_metrics


lat1,lng1,lat2,lng2,distance_km,distance_m
f64,f64,f64,f64,f64,f64
40.7128,-74.006,42.3601,-71.0589,306.204589,306204.588543


# Computing H3 Cell Area

We can also compute the area of the H3 cell in km² or m².


In [21]:
df_area = pl.DataFrame({"h3_resolution": list(range(0, 16))}).with_columns(
    area_km2=plh3.average_hexagon_area("h3_resolution", "km^2"),
    num_cells=plh3.get_num_cells("h3_resolution"),
)

df_area


h3_resolution,area_km2,num_cells
i64,f64,i64
0,4.3574e6,122
1,609788.441794,842
2,86801.780398,5882
3,12393.434655,41162
4,1770.347654,288122
…,…,…
11,0.002149,237279209162
12,0.000307,1660954464122
13,0.000043,11626681248842


# Conclusion

This notebook provided a brief introduction to `polars_h3`. We covered:

- Converting between lat/lng and H3 cells
- Checking validity and resolution
- Finding neighbors and directed edges
- Computing distances and areas

For more advanced use cases, refer to the documentation and the test files included with `polars_h3`.
